In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
from bs4 import BeautifulSoup

In [4]:
from msedge.selenium_tools import Edge, EdgeOptions

In [5]:
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

In [6]:
url = "https://www.amazon.com"
driver.get(url)

In [7]:
def  get_url(search_term):
    """Generate a url fro search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1"
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [8]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&ref=nb_sb_noss_1


In [9]:
driver.get(url)

## Extract the Collection

In [10]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [11]:
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [12]:
len(results)

22

### Prototype the record

In [13]:
item = results[0]

In [14]:
atag = item.h2.a

In [15]:
atag.text

'SAMSUNG 34-Inch Odyssey G5 Ultra-Wide Gaming Monitor with 1000R Curved Screen, 165Hz, 1ms, FreeSync Premium, WQHD (LC34G55TWWNXZA, 2020 Model), Black '

In [16]:
url = 'https://www.amazon.com' + atag.get('href')

In [17]:
price_parent = item.find('span', 'a-price')

In [18]:
price = price_parent.find('span', 'a-offscreen').text

In [21]:
rating = item.i.text

In [22]:
review_count = item.find('span', {'class': 'a-size-base'}).text

In [25]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    price_parent = item.find('span', 'a-price') 
    price = price_parent.find('span', 'a-offscreen').text
    
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base'}).text
    
    result = (description, price, rating, review_count, url)
    return result


In [26]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for items in results:
    records.append(extract_record(item))

### Error Handling

In [31]:
def extract_record(item):
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price') 
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return 
        
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    return result

In [32]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [33]:
records[0]

('SAMSUNG 34-Inch Odyssey G5 Ultra-Wide Gaming Monitor with 1000R Curved Screen, 165Hz, 1ms, FreeSync Premium, WQHD (LC34G55TWWNXZA, 2020 Model), Black',
 '$599.99',
 '4.6 out of 5 stars',
 '3,543',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A04961222KVJJGI2X8D5E&url=%2FSAMSUNG-34-Inch-Ultra-Wide-FreeSync-LC34G55TWWNXZA%2Fdp%2FB08MVBYWGQ%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1634637667%26sr%3D8-1-spons%26psc%3D1&qualifier=1634637667&id=4397828950984473&widgetName=sp_atf')

In [34]:
records[1]

('SAMSUNG 34 Inch Ultrawide QHD Curved Monitor, 100hz, 1440p Monitor, Computer Monitor, 1000R Curved, USB-C, HDR10, Vertical Monitor, TUV-Certified Intelligent Eye Care, S65UA (LS34A650UXNXGO)',
 '$649.99',
 '4.3 out of 5 stars',
 '33',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A001299824R6GD09EHQKN&url=%2FSAMSUNG-Adjustable-TUV-Certified-Intelligent-LS34A650UXNXGO%2Fdp%2FB08V71HXY3%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1634637667%26sr%3D8-2-spons%26psc%3D1&qualifier=1634637667&id=4397828950984473&widgetName=sp_atf')

In [35]:
records[2]

('Lenovo G34w-10 34-Inch WQHD Curved Gaming Monitor, 21:9 Ultra-Wide, 3 Side NearEdgeless, 3440 x 1440, AMD FreeSync, 144Hz, HDMI, DP, Adjustable Stand, VESA, 66A1GCCBUS, Black',
 '$428.00',
 '4.4 out of 5 stars',
 '388',
 'https://www.amazon.com/Lenovo-G34w-10-34-Inch-Curved-Monitor/dp/B084R2XYM9/ref=sr_1_5?dchild=1&keywords=ultrawide+monitor&qid=1634637667&sr=8-5')

In [36]:
records[5]

('Philips Computer Monitors 292E2E 29"" Frameless IPS Monitor, UltraWide Full HD 2560x1080, 126% sRGB/110% NTSC, 75Hz FreeSync, Height Adjustable, VESA, 4Yr Advance Replacement, Black',
 '$239.99',
 '4.6 out of 5 stars',
 '9,960',
 'https://www.amazon.com/Philips-Computer-Monitors-292E2E-Replacement/dp/B08KFLL9JW/ref=sr_1_12?dchild=1&keywords=ultrawide+monitor&qid=1634637667&sr=8-12')

In [37]:
for row in records:
    print(row[1])

$599.99
$649.99
$428.00
$249.96
$397.97
$239.99
$449.99
$429.99
$309.99
$115.99
$247.89
$399.99
$399.99
$319.45
$262.01


### Getting on next page

In [38]:
def  get_url(search_term):
    """Generate a url fro search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1"
    search_term = search_term.replace(' ', '+')
    
    ## add term query to it.
    url = template.format(search_term)
    ## add page query 
    url += '&page{}'
    
    return url

### Putting all pages together

In [60]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

def  get_url(search_term):
    """Generate a url fro search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_1"
    search_term = search_term.replace(' ', '+')
    
    ## add term query to it.
    url = template.format(search_term)
    ## add page query 
    url += '&page={}'
    
    return url

def extract_record(item):
    """extract and return data for a single record"""
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price') 
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return 
        
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    return result


def main(search_term):
    """Run main program"""
    # start web driver
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    
    driver.close() 
    
    ## save data in CSV file
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'url'])
        writer.writerows(records)

In [61]:
main('ultrawide monitor')